# Saliency Visualization

Helpful Links:
https://fairyonice.github.io/Saliency-Map-with-keras-vis.html
https://www.machinecurve.com/index.php/2019/11/25/visualizing-keras-cnn-attention-saliency-maps/#introducing-keras-vis

In [ ]:
!pip3 install git+git://github.com/raghakot/keras-vis.git --upgrade --no-deps

In [ ]:
!pip3 install -r requirements.txt

import keras
import vis

from keras import activations
from keras.models import load_model
from keras.utils import to_categorical, CustomObjectScope

import matplotlib.pyplot as plt
import matplotlib.patches as patches

from models import AlphaNet

import numpy as np
from numpy import loadtxt

from tools import *
from train import *

from vis.visualization import visualize_saliency
from vis.utils import utils


In [ ]:
assert int(tf.__version__[0]) >= 2
from skimage.transform import resize

In [ ]:
# load model
custom_metrics = {
    "top3_accuracy": top3_acc,
    "top5_accuracy": top5_acc
}
model = load_model('../BravoNet.h5', custom_objects=custom_metrics) 
model.summary()


In [ ]:
X_train, y_train = load_data()
X_val, y_val = load_data("val")
X_test, _ = load_data("test")

y_train = to_categorical(y_train, 200)
y_val = to_categorical(y_val, 200)

In [ ]:
boxes = get_box_dict_val()
assert(len(boxes) == len(X_val))


In [ ]:
labels = get_label_dict()
words = get_word_labels()

test_images = os.listdir("data/tiny-imagenet-200/test/images/")
assert len(X_test) == len(test_images)

In [ ]:
with CustomObjectScope({"top3_accuracy": top3_acc, "top5_accuracy": top5_acc,}):
    layer_index = -1
    model.layers[layer_index].activation = activations.linear
    model = utils.apply_modifications(model)  

In [ ]:
# indices_to_visualize = [ 5, 19, 38, 82, 111, 76, 194 ]
indices_to_visualize = np.random.choice(len(X_val), 10)

with CustomObjectScope({"top3_accuracy": top3_acc, "top5_accuracy": top5_acc,}):

    for index_to_visualize in indices_to_visualize:
        # Get input
        input_image = X_val[index_to_visualize]
        input_class = np.argmax(y_val[index_to_visualize])
        
        # Matplotlib preparations
        fig, axes = plt.subplots(1, 2)
        
        # Generate visualization
        input_image = resize(input_image, (72, 72, 3))
        visualization = visualize_saliency(model, layer_index, filter_indices=input_class, seed_input=input_image)

        y_pred = model.predict(input_image[np.newaxis,...])
        class_idxs_sorted = np.argsort(y_pred.flatten())[::-1]

        class_idx = class_idxs_sorted[0]

        axes[0].imshow(input_image) 
        b = boxes['val_'+str(index_to_visualize)+'.JPEG']
        rect = patches.Rectangle((int(b[0]),int(b[1])),int(b[2]),int(b[3]),linewidth=1,edgecolor='b',facecolor='none')
        axes[0].add_patch(rect)
        axes[0].set_title('Original image')
        axes[1].imshow(visualization)

        axes[1].set_title('Saliency map')
        fig.suptitle(words[labels[class_idx]])
        fname = './saliency'+str(index_to_visualize)+'.png'
        plt.savefig(fname, bbox_inches='tight')
        plt.show()


        